In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

## The Advanced Computing Center for Research and Education

**Project Overview**
The Advanced Computing Center for Research and Education (ACCRE) operates Vanderbilt University's high-performance computing cluster. Jobs submitted to ACCRE are managed by the [slurm scheduler](https://slurm.schedmd.com/documentation.html), which tracks compute and memory resources.

ACCRE staff have hypothesized that the scheduler sometimes becomes unresponsive because it is processing large bursts of job completions. This especially affects automated job submitters, such as members of the Open Science Grid.

Your goal is to evaluate whether the data supports the hypothesis of bursts of job completions contributing to scheduler unresponsiveness.

You are provided three datasets:  
* fullsample.csv: Contains slurm job records. Job completions correspond to jobs in the "COMPLETED" state with exit code "0:0".  
* slurm_wrapper_ce5.log, slurm_wrapper_ce6.log: These log files contain every slurm command executed by the CE5 and CE6 servers (gateways to the Open Science Grid).  
Unresponsive periods are indicated by "sbatch" commands from user 9204 that have:  
    * return code = 1
    * execution time > 15 seconds

**Phase 1: Explore the Data**  
Objectives:  
* Understand the purpose of each dataset.  
* Inspect column types, sizes, and example rows.  

Notebook Sections:  
* Code: Load each dataset, preview rows, summarize columns.  
* Markdown: Notes on data quality and initial observations.  

In [2]:
# Read csv file
jobs = pd.read_csv('../data/fullsample.csv')

In [3]:
# Read log file
ce5 = pd.read_csv('../data/slurm_wrapper_ce5.log',
                  header=None,
                  delimiter=' - ',
                  engine='python')

In [4]:
# Read log file
ce6 = pd.read_csv('../data/slurm_wrapper_ce6.log',
                  header=None,
                  delimiter=' - ',
                  engine='python')

> ## Explore Data
>
> Below we examine the types of data and create a table with a description of each element in the jobs dataframe. The main issue we are adressing in this project is incomplete jobs and what is likley causing these jobs to be incomplete. The first thing we need to do is calculate how many of these jobs are incomplete. Below, I have added an additional table defining each variable in the dataframe.
>
>* **0.28%** of the jobs in our dataset are **not completed.**
>* There are **20,801** incomplete jobs total in our dataframe.

In [7]:
# Define variables in jobs dataframe

column | description
-------|---------
JOBID | The identification number of the job or job step. Array jobs are in the form ArrayJobID_ArrayTaskID
STATE | Job state or status (COMPLETED, CANCELLED, FAILED, TIMEOUT, PREEMPTED, etc.)
BEGIN | Beginning time for the job.
END | Ending time for the job.
REQMEM | Requested memory in megabytes. May be per-core (Mc) or per-node (Mn)
USEDMEM | Used memory in megabytes per-node
REQTIME | Requested time in d-hh:mm:ss or hh:mm:ss
USEDTIME | Used time in d-hh:mm:ss or hh:mm:ss
NODES | Number of servers used for this job
CPUS | Total number of CPU-cores allocated to the job
PARTITION | Identifies the partition on which the job ran.
EXITCODE | The exit code returned by the job script or salloc, typically as set by the exit() function. Following the colon is the signal that caused the process to terminate if it was terminated by a signal.

In [6]:
# Inspect variable types
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7395885 entries, 0 to 7395884
Data columns (total 12 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   JOBID      object
 1   STATE      object
 2   BEGIN      object
 3   END        object
 4   REQMEM     object
 5   USEDMEM    object
 6   REQTIME    object
 7   USEDTIME   object
 8   NODES      int64 
 9   CPUS       int64 
 10  PARTITION  object
 11  EXITCODE   object
dtypes: int64(2), object(10)
memory usage: 677.1+ MB


In [8]:
# Find incomplete jobs
jobs['STATE'].describe()

count       7395885
unique          145
top       COMPLETED
freq        7375084
Name: STATE, dtype: object

In [9]:
# Calculate sample statistic
print(f'{(((7395885 - 7375084) / 7395885) * 100):.2f}% of the jobs in our dataset are not completed.')

0.28% of the jobs in our dataset are not completed.


> ## Clean and Sort Data
>
> We have analyzed the "jobs" csv file and gained an understanding of the data. We have two other dataframes we want to better understand. The log files we have imported are messy and need some adjustment. Below, we will rename each column and identify its purpose. This will give us a better idea of how these log files can help us find the main problem in our data.
> 
>
>* **Rename columns** with descriptive indicators.
>* Eliinate unnecesary information.
>* Create seperate **date and time** columns.
>
>

In [10]:
ce5.head(2)

,0,1,2,3,4,5
0,2020-10-16 08:15:39.278699,user 0,retry 0,time 0.07347559928894043,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
1,2020-10-16 08:18:08.313309,user 0,retry 0,time 0.18363237380981445,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."


In [11]:
ce5 = pd.DataFrame(ce5)

# Rename columns
ce5 = ce5.rename(
    columns={
        0: 'date_time',
        1: 'user',
        2: 'retry',
        3: 'runtime',
        4: 'return_code',
        5: 'command'
    }
)

In [12]:
# Clean and strip column values
ce5['user'] = ce5['user'].str.replace(r'^user\s*', '', regex=True)
ce5['retry'] = ce5['retry'].str.replace(r'^retry\s*', '', regex=True)
ce5['runtime'] = ce5['runtime'].str.replace(r'^time\s*', '', regex=True)
ce5['return_code'] = ce5['return_code'].str.replace(r'^returncode\s*', '', regex=True)
ce5['command'] = ce5['command'].str.replace(r'^command\s*', '', regex=True)

In [13]:
# create two seperate date and time columns
ce5['date_time'] = pd.to_datetime(ce5['date_time'],
                                 format = 'mixed',
                                 yearfirst = True
)

ce5['date'] = ce5['date_time'].dt.date
ce5['time'] = ce5['date_time'].dt.time

In [14]:
ce6 = pd.DataFrame(ce6)

# Rename columns
ce6 = ce6.rename(
    columns={
        0: 'date_time',
        1: 'user',
        2: 'retry',
        3: 'runtime',
        4: 'return_code',
        5: 'command'
    }
)

In [15]:
# Clean and strip column values
ce6['user'] = ce5['user'].str.replace(r'^user\s*', '', regex=True)
ce6['retry'] = ce5['retry'].str.replace(r'^retry\s*', '', regex=True)
ce6['runtime'] = ce5['runtime'].str.replace(r'^time\s*', '', regex=True)
ce6['return_code'] = ce5['return_code'].str.replace(r'^returncode\s*', '', regex=True)
ce6['command'] = ce5['command'].str.replace(r'^command\s*', '', regex=True)

In [17]:
# create two seperate date and time columns
ce6['date_time'] = pd.to_datetime(ce6['date_time'],
                                 format = 'mixed',
                                 yearfirst = True
)

ce6['date'] = ce6['date_time'].dt.date
ce6['time'] = ce6['date_time'].dt.time 

In [19]:
# Convert runtime to numeric type
ce5['runtime'] = ce5['runtime'].astype('float')
ce6['runtime'] = ce6['runtime'].astype('float')

**Phase 2: Clean and Transform the Data**  
Objectives:  
* Extract job completions from fullsample.csv.  
* Parse CE5 and CE6 logs to identify unresponsive events.  
* Create analysis-ready features (time windows, completion counts, unresponsiveness indicators).  
* Optionally include other features (currently running jobs or resource usage, time-of-day).  

Notebook Sections:  
* Code: Filtering and transforming datasets.  
* Markdown: Document preprocessing steps and reasoning.  
* Code: Combine datasets into a single dataset suitable for analysis.

In [20]:
# Extract completed jobs
complete_jobs = jobs[jobs['STATE'] == 'COMPLETED']

In [21]:
ce5.head()

,date_time,user,retry,runtime,return_code,command,date,time
0,2020-10-16 08:15:39.278699,0,0,0.073476,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",2020-10-16,08:15:39.278699
1,2020-10-16 08:18:08.313309,0,0,0.183632,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",2020-10-16,08:18:08.313309
2,2020-10-16 08:22:48.128689,0,0,0.075471,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",2020-10-16,08:22:48.128689
3,2020-10-16 08:25:13.257408,0,0,0.094844,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",2020-10-16,08:25:13.257408
4,2020-10-16 08:31:01.460723,0,0,0.074988,0,"['/usr/bin/sacct', '-u', 'appelte1', '-S', '20...",2020-10-16,08:31:01.460723


For this project, we are interested in jobs from user 9204 (the test user) where the command starts with '/usr/bin/squeue', the returncode is non-zero and the time is greater than 15. These conditions indicate that the scheduler becaem unresponsive at that point in time.

**Phase 3: Analyze and Visualize**  
Objectives:  
* Explore the relationship between job completions and unresponsiveness.  
* Create visualizations and basic summary statistics.  

Notebook Sections:  
* Code: Time-series plots, scatterplots, boxplots, summary statistics.
* Markdown: Interpret the visualizations and describe patterns.  
* Code: Fit a simple logistic regression to test the hypothesis.
* Markdown: Summarize the results and draw conclusions from the model.  
* Optional: Explore additional factors (eg. day of week).

**Phase 4: Interpret and Conclude**  
Objectives:  
* Answer the main question: Does the data support the hypothesis that the slurm scheduler is more likely to be unresponsive during bursts of job completions?  
* Summarize findings and limitations.  

Notebook Sections:    
* Markdown: Summarize evidence for or against the hypothesis.  
* Markdown: Provide a clear conclusion.  

**Final Deliverable:**
A single Jupyter notebook that includes:  
1. Introduction & dataset overview  
2. Data exploration & cleaning  
3. Feature engineering  
4. Analysis & visualizations  
5. Interpretation & conclusion